In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
from huggingface_hub import login
login()

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_name = "mistralai/Mistral-Nemo-Instruct-2407"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)

tokenizer_config.json:   0%|          | 0.00/181k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.26M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/622 [00:00<?, ?B/s]

2025-09-24 16:53:43.042698: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1758732823.408017      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1758732823.506372      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


...

model-00005-of-00005.safetensors:   0%|          | 0.00/4.91G [00:00<?, ?B/s]

model-00004-of-00005.safetensors:   0%|          | 0.00/4.91G [00:00<?, ?B/s]

model-00003-of-00005.safetensors:   0%|          | 0.00/4.91G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [3]:
def generate_text(prompt, max_length=2000, num_return_sequences=1):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_length=max_length,
        num_return_sequences=num_return_sequences,
        do_sample=True,
        top_k=50,
        top_p=0.95,
        temperature=0.7,
    )
    return [tokenizer.decode(output, skip_special_tokens=True) for output in outputs][0]

In [4]:
from langchain.llms.base import LLM
from typing import Any, Dict, List

class CustomHFLLM(LLM):
    def _call(self, prompt: str, stop: Any = None) -> str:
        return generate_text(prompt)
    
    @property
    def _llm_type(self) -> str:
        return "custom_huggingface"

llm = CustomHFLLM()

In [5]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

# ====== Agent1 for Research Agent ======
research_prompt = PromptTemplate(
    input_variables=["topic"],
    template=(
        "You are a specialized research agent. Conduct comprehensive research about: {topic}. "
        "Provide accurate and up-to-date information including key points, important statistics, "
        "and recent trends in this field. Focus on providing actionable and relevant information."
    )
)
research_agent = LLMChain(llm=llm, prompt=research_prompt, output_key="research_data")

# ====== Agent2 for Summarizer Agent ======
summarizer_prompt = PromptTemplate(
    input_variables=["research_data"],
    template=(
        "You are a professional summarization agent. Analyze the following research data and provide a clear and concise summary: \n"
        "{research_data}\n\n"
        "Focus on extracting key points and core ideas and organizing them in a logical way."
    )
)
summarizer_agent = LLMChain(llm=llm, prompt=summarizer_prompt, output_key="summary")

# ====== Agent3 for Reasoning Agent ======
reasoning_prompt = PromptTemplate(
    input_variables=["summary", "topic"],
    template=(
        "You are an analysis and reasoning agent. Based on the following summary about {topic}: \n"
        "{summary}\n\n"
        "Analyze this information and draw logical conclusions. "
        "Identify patterns, relationships, and implications of this information."
    )
)
reasoning_agent = LLMChain(llm=llm, prompt=reasoning_prompt, output_key="analysis")

# ====== Agent4 for Decision Agent ======
decision_prompt = PromptTemplate(
    input_variables=["research_data", "summary", "analysis", "topic"],
    template=(
        "You are a strategic decision-making agent. Based on research about {topic}, provide "
        "practical and actionable recommendations. Identify the best courses of action, "
        "potential risks, and available opportunities."
    )
)
decision_agent = LLMChain(llm=llm, prompt=decision_prompt, output_key="recommendations")

/tmp/ipykernel_36/3804247446.py:13: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  research_agent = LLMChain(llm=llm, prompt=research_prompt, output_key="research_data")


In [6]:
#  coordination system between agents
def run_system(topic: str):
    print(f"Requested Topic: {topic}")
    print("\n" + "="*60)
    
    # Phase 1: Initial research
    print("\n[🧠 Research Agent Working...]")
    research_data = research_agent.run(topic)
    print(f"\nResearch Results:\n{research_data.strip()}")
    
    print("\n" + "="*60)
    
    # Phase 2: Summarization and analysis (could happen in parallel with sufficient resources)
    print("\n[📊 Summarization Agent Working...]")
    summary = summarizer_agent.run(research_data=research_data)
    print(f"\nSummary of Results:\n{summary.strip()}")
    
    print("\n" + "="*60)
    print("\n[🔍 Reasoning Agent Working...]")
    analysis = reasoning_agent.run(summary=summary, topic=topic)
    print(f"\nAnalysis and Reasoning:\n{analysis.strip()}")
    
    print("\n" + "="*60)
    
    # Phase 3: Decision making based on all inputs
    print("\n[🎯 Decision Agent Working...]")
    recommendations = decision_agent.run({
        "research_data": research_data,
        "summary": summary,
        "analysis": analysis,
        "topic": topic
    })
    print(f"\nRecommendations and Decisions:\n{recommendations.strip()}")
    
    print("\n" + "="*60)
    print("✅ Analysis and recommendation process completed successfully!")
    
    return {
        "research_data": research_data,
        "summary": summary,
        "analysis": analysis,
        "recommendations": recommendations
    }

In [7]:
# run the system on multiple topics
def run_multiple_topics(topics: List[str]):
    results = {}
    for topic in topics:
        print(f"\n{'#'*70}")
        print(f"Processing Topic: {topic}")
        print(f"{'#'*70}")
        result = run_system(topic)
        if result:
            results[topic] = result
    return results

In [9]:
# Run the system
if __name__ == "__main__":
    # Topics to analyze
    topics = [
        "Artificial intelligence and its impact on the job market"
    ]
    
    all_results = run_multiple_topics(topics)

/tmp/ipykernel_202/4110894409.py:8: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  research_data = research_agent.run(topic)
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



######################################################################
Processing Topic: Artificial intelligence and its impact on the job market
######################################################################
Requested Topic: Artificial intelligence and its impact on the job market


[🧠 Research Agent Working...]


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



Research Results:
You are a specialized research agent. Conduct comprehensive research about: Artificial intelligence and its impact on the job market. Provide accurate and up-to-date information including key points, important statistics, and recent trends in this field. Focus on providing actionable and relevant information. Be concise and structured.

**Artificial Intelligence and the Job Market: A Comprehensive Analysis**

**1. Key Points**

   - **Automation and Job Displacement**: AI and automation can automate repetitive tasks, potentially displacing jobs in sectors like manufacturing, customer service, and data entry.
   - **Job Creation**: While AI may automate certain jobs, it also creates new ones. These include roles like AI specialists, data scientists, and machine learning engineers.
   - **Job Transformation**: Many jobs will change rather than disappear. AI can augment human capabilities, leading to new skills requirements and changes in job descriptions.
   - **Skills

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



Summary of Results:
You are a professional summarization agent. Analyze the following research data and provide a clear and concise summary: 
You are a specialized research agent. Conduct comprehensive research about: Artificial intelligence and its impact on the job market. Provide accurate and up-to-date information including key points, important statistics, and recent trends in this field. Focus on providing actionable and relevant information. Be concise and structured.

**Artificial Intelligence and the Job Market: A Comprehensive Analysis**

**1. Key Points**

   - **Automation and Job Displacement**: AI and automation can automate repetitive tasks, potentially displacing jobs in sectors like manufacturing, customer service, and data entry.
   - **Job Creation**: While AI may automate certain jobs, it also creates new ones. These include roles like AI specialists, data scientists, and machine learning engineers.
   - **Job Transformation**: Many jobs will change rather than dis

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



Analysis and Reasoning:
You are an analysis and reasoning agent. Based on the following summary about Artificial intelligence and its impact on the job market: 
You are a professional summarization agent. Analyze the following research data and provide a clear and concise summary: 
You are a specialized research agent. Conduct comprehensive research about: Artificial intelligence and its impact on the job market. Provide accurate and up-to-date information including key points, important statistics, and recent trends in this field. Focus on providing actionable and relevant information. Be concise and structured.

**Artificial Intelligence and the Job Market: A Comprehensive Analysis**

**1. Key Points**

   - **Automation and Job Displacement**: AI and automation can automate repetitive tasks, potentially displacing jobs in sectors like manufacturing, customer service, and data entry.
   - **Job Creation**: While AI may automate certain jobs, it also creates new ones. These include r

In [8]:
NGROK_TOKEN = ""
API_KEY = ""

In [9]:
!pip install pyngrok

In [10]:
from fastapi import FastAPI, Request, HTTPException
import uvicorn, threading, time, socket
from pyngrok import ngrok, conf

In [15]:
app = FastAPI()

@app.post("/generate")
async def gen(req: Request):
    if req.headers.get("authorization") != f"Bearer {API_KEY}":
        raise HTTPException(status_code=401, detail="Unauthorized")
    data = await req.json()
    return {
        "response": generate_text(
            data.get("prompt", ""),
            data.get("max_length", 1000)
        )
    }

@app.post("/analyze")
async def analyze_topic(req: Request):
    if req.headers.get("authorization") != f"Bearer {API_KEY}":
        raise HTTPException(status_code=401, detail="Unauthorized")
    
    data = await req.json()
    topic = data.get("topic", "")
    
    
    research_data = research_agent.run(topic)
    summary = summarizer_agent.run(research_data=research_data)
    analysis = reasoning_agent.run(summary=summary, topic=topic)
    recommendations = decision_agent.run({
        "research_data": research_data,
        "summary": summary,
        "analysis": analysis,
        "topic": topic
    })
    
    # Return the final result 
    return {
        "topic": topic,
        "result": recommendations
    }

In [16]:
def free_port():
    s = socket.socket()
    s.bind(('', 0))
    port = s.getsockname()[1]
    s.close()
    return port

port = free_port()
if NGROK_TOKEN:
    conf.get_default().auth_token = NGROK_TOKEN
    public_url = ngrok.connect(port).public_url
    print("Your public URL:", public_url)

def run(): 
    uvicorn.run(app, host="0.0.0.0", port=port)
threading.Thread(target=run, daemon=True).start()
time.sleep(1)

Your public URL: https://456c5a3bd3ab.ngrok-free.app                                                


INFO:     Started server process [36]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:51765 (Press CTRL+C to quit)


INFO:     35.185.209.55:0 - "POST /analyze/analyze HTTP/1.1" 404 Not Found
INFO:     35.185.209.55:0 - "POST /analyze/analyze HTTP/1.1" 404 Not Found
INFO:     35.185.209.55:0 - "POST /analyze/analyze HTTP/1.1" 404 Not Found
INFO:     35.185.209.55:0 - "POST /analyze/analyze HTTP/1.1" 404 Not Found


test

In [11]:
app = FastAPI(title="Multi-Agent AI System")

# Endpoint 1: Direct text generation from the model
@app.post("/generate")
async def generate_direct(req: Request):
    if req.headers.get("authorization") != f"Bearer {API_KEY}":
        raise HTTPException(status_code=401, detail="Unauthorized")
    
    data = await req.json()
    prompt = data.get("prompt", "")
    max_length = data.get("max_length", 1000)
    
    response_text = generate_text(prompt, max_length)
    
    return {
        "prompt": prompt,
        "generated_text": response_text
    }

# Endpoint 2: Full multi-agent analysis pipeline
@app.post("/analyze")
async def analyze_topic(req: Request):
    if req.headers.get("authorization") != f"Bearer {API_KEY}":
        raise HTTPException(status_code=401, detail="Unauthorized")
    
    data = await req.json()
    topic = data.get("topic", "")
    
    if not topic:
        raise HTTPException(status_code=400, detail="Topic is required")
    
    # Run the full multi-agent pipeline
    research_data = research_agent.run(topic)
    summary = summarizer_agent.run(research_data=research_data)
    analysis = reasoning_agent.run(summary=summary, topic=topic)
    recommendations = decision_agent.run({
        "research_data": research_data,
        "summary": summary,
        "analysis": analysis,
        "topic": topic
    })
    
    # Return comprehensive results
    return {
        "topic": topic,
        "research_data": research_data,
        "summary": summary,
        "analysis": analysis,
        "recommendations": recommendations
    }

# Health check endpoint
@app.get("/")
async def health_check():
    return {"status": "active", "service": "Multi-Agent AI System"}

@app.get("/health")
async def health():
    return {"status": "healthy"}

In [12]:
def free_port():
    s = socket.socket()
    s.bind(('', 0))
    port = s.getsockname()[1]
    s.close()
    return port

port = free_port()

if NGROK_TOKEN:
    conf.get_default().auth_token = NGROK_TOKEN
    public_url = ngrok.connect(port).public_url
    print("🌐 Your public URL:", public_url)
    print("🔑 API Key:", API_KEY)
    print("\n📋 Available endpoints:")
    print(f"   • POST {public_url}/generate")
    print(f"   • POST {public_url}/analyze")
    print(f"   • GET  {public_url}/health")

def run(): 
    uvicorn.run(app, host="0.0.0.0", port=port)

threading.Thread(target=run, daemon=True).start()
time.sleep(3)

print("\n✅ Server is running! Use the endpoints above to interact with the system.")

🌐 Your public URL: https://1d0f745acd0e.ngrok-free.app                                             
🔑 API Key: secret123

📋 Available endpoints:
   • POST https://1d0f745acd0e.ngrok-free.app/generate
   • POST https://1d0f745acd0e.ngrok-free.app/analyze
   • GET  https://1d0f745acd0e.ngrok-free.app/health


INFO:     Started server process [36]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:40497 (Press CTRL+C to quit)



✅ Server is running! Use the endpoints above to interact with the system.
INFO:     34.19.100.134:0 - "GET /health HTTP/1.1" 200 OK
INFO:     34.19.100.134:0 - "GET /health HTTP/1.1" 200 OK


/tmp/ipykernel_36/1663875307.py:33: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  research_data = research_agent.run(topic)
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


INFO:     34.19.100.134:0 - "POST /analyze HTTP/1.1" 200 OK
INFO:     34.19.100.134:0 - "GET /health HTTP/1.1" 200 OK
INFO:     34.19.100.134:0 - "GET /health HTTP/1.1" 200 OK
INFO:     34.19.100.134:0 - "GET /health HTTP/1.1" 200 OK


...

INFO:     34.19.100.134:0 - "POST /analyze HTTP/1.1" 200 OK
INFO:     34.19.100.134:0 - "GET /health HTTP/1.1" 200 OK
INFO:     34.19.100.134:0 - "GET /health HTTP/1.1" 200 OK


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


INFO:     34.19.100.134:0 - "POST /generate HTTP/1.1" 200 OK
INFO:     34.19.100.134:0 - "GET /health HTTP/1.1" 200 OK
INFO:     34.19.100.134:0 - "GET /health HTTP/1.1" 200 OK


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


INFO:     34.19.100.134:0 - "POST /analyze HTTP/1.1" 200 OK
INFO:     34.19.100.134:0 - "GET /health HTTP/1.1" 200 OK
INFO:     34.19.100.134:0 - "GET /health HTTP/1.1" 200 OK
INFO:     34.19.100.134:0 - "GET /health HTTP/1.1" 200 OK
